In [129]:
import pandas as pd
import requests

In [130]:
portfolio = pd.read_csv('portfolio.csv')
portfolio['Дата'] = portfolio['Дата'].astype('datetime64[ns]')
transactions = pd.read_csv('transactions.csv').drop_duplicates(['Дата заключения', 'Время заключения', 'Статус'])
transactions['Дата заключения'] = pd.to_datetime(transactions['Дата заключения'] + ' ' + transactions['Время заключения'], dayfirst=True)
transactions['Дата расчетов'] = pd.to_datetime(transactions['Дата расчетов'], dayfirst=True)
transactions = transactions.drop('Время заключения', axis=1).dropna(axis=1).sort_values('Дата заключения').reset_index(drop=True)

In [131]:
transactions

,Дата заключения,Дата расчетов,Наименование,Код,Валюта,Вид,Количество,Цена,Сумма,НКД,Комиссия Брокера,Комиссия Биржи,Номер сделки,Статус
0,2021-05-17 10:21:16,2021-05-19,SBSPETF,SBSP,RUB,Покупка,1,1633.000,1633.0,0.0,0.98,0.15,3943134134,О
1,2021-05-17 10:21:16,2021-05-19,SBSPETF,SBSP,RUB,Покупка,1,1633.000,1633.0,0.0,0.98,0.15,3943134134,И
2,2021-05-17 10:21:16,2021-05-19,SBSPETF,SBSP,RUB,Покупка,1,1633.000,1633.0,0.0,0.98,0.15,3943134134,ЗО
3,2021-05-17 10:22:48,2021-05-19,FXUSETF,FXUS,RUB,Покупка,1,5615.000,5615.0,0.0,3.37,0.52,3943142919,И
4,2021-05-17 10:22:48,2021-05-19,FXUSETF,FXUS,RUB,Покупка,1,5615.000,5615.0,0.0,3.37,0.52,3943142919,О
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
262,2023-06-23 10:33:59,2023-06-27,МТС-ао,MTSS,RUB,Покупка,10,343.350,3433.5,0.0,2.06,1.03,7792729150,О
263,2023-07-18 16:07:57,2023-07-20,СевСт-ао,CHMF,RUB,Покупка,1,1275.600,1275.6,0.0,0.77,0.38,7908972071,И
264,2023-07-18 17:52:50,2023-07-20,Сбербанк,SBER,RUB,Покупка,10,246.210,2462.1,0.0,1.48,0.73,7909781928,И
265,2023-07-18 17:55:20,2023-07-20,Системаао,AFKS,RUB,Покупка,100,17.744,1774.4,0.0,1.06,0.54,7909796018,И


In [132]:
transactions_executed = transactions[transactions['Статус'] == 'И']

In [133]:
ticker_list = list(transactions_executed['Код'].unique())
share_split_dict = {
    'FXGD': ['2022-02-17', 10],
    'SBMX': ['2021-06-09', 100],
    'FXUS': ['2022-01-24', 100],
    'FXRL': ['2021-11-24', 100],
    'FXRU': ['2022-02-17', 10],
    'FXDE': ['2021-12-15', 100]
}
replacement_dict = {
    'VTBE': 'RSHE',
    'RU000A101FA1': 'SU25084RMFS3'
}

In [134]:
def fix_split():
    ticker_list_copy = ticker_list.copy()
    for ticker in ticker_list_copy:
        if ticker == 'RU000A101FA1':
            new_ticker = 'SU25084RMFS3'
            ticker_list.remove(ticker)
            ticker_list.append(new_ticker)
            transactions_executed.loc[transactions['Код'] == ticker, 'Код'] = new_ticker
        if ticker == 'VTBE':
            new_ticker = 'RSHE'
            ticker_list.remove(ticker)
            ticker_list.append(new_ticker)
            transactions_executed.loc[transactions['Код'] == ticker, 'Код'] = new_ticker
    for ticker in ticker_list:
        if ticker in share_split_dict:
            transactions_executed.loc[(transactions_executed['Код'] == ticker) & (transactions_executed['Дата заключения'] < share_split_dict[ticker][0]), 'Количество'] = \
            transactions_executed.loc[(transactions_executed['Код'] == ticker) & (transactions_executed['Дата заключения'] < share_split_dict[ticker][0]), 'Количество'] * \
            share_split_dict[ticker][1]
fix_split()

In [135]:
def get_share_price_dict(tickers: list):
    share_price_dict = {}
    for ticker in tickers:
        if ticker in ['SBMX', 'AFKS']:
            round_numb = 3
        elif ticker in ['IRAO']:
            round_numb = 4
        else:
            round_numb = 2
        share = transactions_executed[transactions_executed['Код'] == ticker]
        share_price = ((share['Сумма'].sum() + share['Комиссия Брокера'].sum() + share['Комиссия Биржи'].sum()) / 
                            (share.loc[share['Вид'] == 'Покупка', 'Количество'].sum() - share.loc[share['Вид'] == 'Продажа', 'Количество'].sum())).round(round_numb)
        share_price_dict[ticker] = share_price
    return share_price_dict

In [136]:
share_price_dict = get_share_price_dict(ticker_list)

In [137]:
def get_share_amount_dict(tickers: list):
    share_amount_dict = {}
    for ticker in tickers:
        share_amount_dict[ticker] = transactions_executed.loc[transactions_executed['Код'] == ticker, 'Количество'].sum()
    return share_amount_dict

In [138]:
share_amount_dict = get_share_amount_dict(ticker_list)

In [139]:
def get_trading_dict():
    trading_dict = {}
    trading_mode_list = ['TQBR', 'TQTF', 'TQCB', 'TQOB']
    for id_trading in trading_mode_list:
        stocks = 'shares'
        if id_trading in ['TQOB', 'TQCB']:
            stocks = 'bonds'
        url = (f"https://iss.moex.com/iss/engines/stock/markets/{stocks}/boards/{id_trading}/"
            f"securities.csv?iss.meta=off&iss.only=marketdata&marketdata.columns=SECID,LAST")
        csv_text = requests.get(url).text.split('\n')
        trading_dict[id_trading] = csv_text
    return trading_dict

In [140]:
def get_last_prices_dict(tickers: list):
    last_prices_dict = {}
    trading_dict = get_trading_dict()
    for ticker in tickers:
        if ticker == 'RU000A101FA1':
            ticker =  'SU25084RMFS3'
        if ticker == 'VTBE':
            ticker = 'RSHE'
        for value in trading_dict.values():
            for line in value:
                line = line.split(';')
                if ticker in line:
                    last_prices_dict[ticker] = line[1]
    return last_prices_dict

In [141]:
last_prices_dict = get_last_prices_dict(ticker_list)

In [142]:
def get_commission_dict(tickers: list):
    commission_dict = {}
    for ticker in tickers:
        commission_dict[ticker] = (transactions_executed.loc[transactions_executed['Код'] == ticker, 'Комиссия Брокера'] + \
                                     transactions_executed.loc[transactions_executed['Код'] == ticker, 'Комиссия Биржи']) \
                                    .sum().round(2)
    return commission_dict

In [143]:
commission_dict = get_commission_dict(ticker_list)

In [144]:
data = {
    'Котировки': last_prices_dict,
    'Количество': share_amount_dict,
    'Средняя цена': share_price_dict,
    'Комиссия': commission_dict
}
portfolio_main = pd.DataFrame.from_dict(data)
portfolio_main

,Котировки,Количество,Средняя цена,Комиссия
SBSP,,5,1724.6300,2.76
FXUS,,350,57.1700,13.86
FXRL,,192,39.8200,5.28
FXDM,,92,76.6400,3.52
FXRU,,43,94.3000,2.81
FXDE,,124,30.8700,2.66
FXIM,,36,90.1200,2.09
SBMX,17.623,830,15.7570,4.98
FXRW,,1135,1.2800,0.78
FXGD,,82,334.2700,4.61


In [145]:
portfolio_main.to_csv('portfolio_main.csv')

In [146]:
portfolio = pd.read_csv('portfolio_main.csv')\
        .rename(columns={'Unnamed: 0': 'Название'})\
        .dropna(axis=0)\
        .set_index('Название')
portfolio

,Котировки,Количество,Средняя цена,Комиссия
Название,,,,
SBMX,17.623,830,15.7570,4.98
SBER,264.120,40,224.6600,8.06
MTSS,288.400,20,300.9000,5.41
GAZP,173.460,30,174.1700,4.71
LKOH,6103.000,1,4821.3300,4.33
MGNT,5657.500,1,4186.2600,3.76
YNDX,2731.000,1,2642.3800,2.38
CHMF,1348.400,1,1276.7500,1.15
AFKS,18.336,100,17.7600,1.60
